# E3 Listening to the customer is a good way to do business

## 🔥 Get a kernel ready

We're going to create a kernel to start off.

In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI services used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))

/Users/johnmaeda/Documents/GitHub/pizzashop-and-ai/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 🏁 Let's start backwards from the customer

The shopkeeper has assembled their strengths, weaknesses, opportunities, and threats. That information is available in "native" code form below:

In [2]:
strength_questions = ["What unique recipes or ingredients does the pizza shop use?","What are the skills and experience of the staff?","Does the pizza shop have a strong reputation in the local area?","Are there any unique features of the shop or its location that attract customers?", "Does the pizza shop have a strong reputation in the local area?", "Are there any unique features of the shop or its location that attract customers?"]
weakness_questions = ["What are the operational challenges of the pizza shop? (e.g., slow service, high staff turnover)","Are there financial constraints that limit growth or improvements?","Are there any gaps in the product offering?","Are there customer complaints or negative reviews that need to be addressed?"]
opportunities_questions = ["Is there potential for new products or services (e.g., catering, delivery)?","Are there under-served customer segments or market areas?","Can new technologies or systems enhance the business operations?","Are there partnerships or local events that can be leveraged for marketing?"]
threats_questions = ["Who are the major competitors and what are they offering?","Are there potential negative impacts due to changes in the local area (e.g., construction, closure of nearby businesses)?","Are there economic or industry trends that could impact the business negatively (e.g., increased ingredient costs)?","Is there any risk due to changes in regulations or legislation (e.g., health and safety, employment)?"]

strengths = [ "Unique garlic pizza recipe that wins top awards","Owner trained in Sicily","Strong local reputation","Prime location on university campus" ]
weaknesses = [ "High staff turnover","Floods in the area damaged the seating areas that are in need of repair","Absence of popular calzones from menu","Negative reviews from younger demographic for lack of hip ingredients" ]
opportunities = [ "Untapped catering potential","Growing local tech startup community","Unexplored online presence and order capabilities","Upcoming annual food fair" ]
threats = [ "Competition from cheaper pizza businesses nearby","There's nearby street construction that will impact foot traffic","Rising cost of cheese will increase the cost of pizzas","No immediate local regulatory changes but it's election season" ]


But instead of just working top down from the owner's viewpoint, it's useful to start from the customer's point of view. And let's assume we interviewed the customer to ask them what they've observed about the pizza shop. We can use the Design Thinking Plugin to better understand the problem at hand. We'll be running it within the kernel so technically speaking we'll be accessing a collection of semantic functions that enable Design Thinking. Note that these ideas were introduced in the second [LinkedIn Learning course for Semantic Kernel](https://www.linkedin.com/learning/building-skills-with-semantic-kernel). 

![](assets/designthinking.png)

# 🦻 Design thinking starts from listening to the customer

Let's use the Semantic Kernel `DesignThinking` plugin to do some useful work for the pizza shop owner.

We have ten observations from customer research that are available to us as `customer_comments`. With these comments in hand, we proceed to categorize the observations, or **empathize** with the customer to start. And then we take that empathy to start and wonder what problem they're experiencing — our goal is to better **define** the problem before trying to solve it.

Let's start with semantic functions that have been encapsulated in a file format that Semantic Kernel uses for direct use in the kernel via a neatly organized folder of subfolders. Note that the prompt template is in the `skprompt.txt` and parameters specific to the function are defined in the `config.json` file.

```directory
plugins-sk/
│
└─── DesignThinking/
     |
     └─── Define/
     |    └─── config.json
     |    └─── skprompt.txt
     |
     └─── Empathize/
          └─── config.json
          └─── skprompt.txt

```

We can source a specific function from within this hierarchy to give to the kernel to process. Let's start with the `Define` function to give you a feel for how that works.

In [6]:
from IPython.display import display, Markdown

pluginsDirectory = "./plugins-sk"

customer_comments = """
Customer 1: The seats look really raggedy.
Customer 2: The garlic pizza is the best on this earth.
Customer 3: I've noticed that there's a new server every time I visit, and they're clueless.
Customer 4: Why aren't there calzones?
Customer 5: I love the garlic pizza and can't get it anywhere else.
Customer 6: The garlic pizza is exceptional.
Customer 7: I prefer a calzone's portable nature as compared with pizza.
Customer 8: Why is the pizza so expensive?
Customer 9: There's no way to do online ordering.
Customer 10: Why is the seating so uncomfortable and dirty?
"""

pluginDT = kernel.import_semantic_skill_from_directory(pluginsDirectory, "DesignThinking");

my_result = await kernel.run_async(pluginDT["Empathize"], input_str=customer_comments)

display(Markdown("## ✨ The categorized observations from the 'Empathize' phase of design thinking\n"+str(my_result)))

## ✨ The categorized observations from the 'Empathize' phase of design thinking
[
    { "sentiment": "Negative", "summary": "Seats are uncomfortable and dirty."},
    { "sentiment": "Positive", "summary": "Garlic pizza is exceptional and the best on earth."},
    { "sentiment": "Negative", "summary": "New servers are clueless."},
    { "sentiment": "Neutral", "summary": "Customers are asking for calzones."},
    { "sentiment": "Negative", "summary": "Pizza is expensive and there's no online ordering."}
]

Note that the customer comments have been analyzed for sentiment and returned in JSON format. It gets more interesting when you chain the result of `Empathize` into the next semantic function `Define` to get a matrix of problem definitions.

In [7]:
my_result = await kernel.run_async(pluginDT["Empathize"], pluginDT["Define"], input_str=customer_comments)

display(Markdown("## ✨ The categorized observations from the 'Empathize' + 'Define' phases of design thinking\n"+str(my_result)))

## ✨ The categorized observations from the 'Empathize' + 'Define' phases of design thinking
| Analysis | Definition | Possible Source |
| --- | --- | --- |
| Uncomfortable and dirty seating | Customers complain about the condition of the seating in the restaurant | Lack of maintenance or cleaning of the seating area |
| Garlic pizza is exceptional and the best on earth | Customers praise the quality of the garlic pizza | High-quality ingredients and skilled chefs |
| New servers are clueless | Customers express frustration with inexperienced servers | Insufficient training or hiring inexperienced staff |
| No calzones available | Customers are disappointed that calzones are not available | Limited menu or supply chain issues |
| Expensive pizza and no online ordering | Customers complain about the high price of pizza and the lack of online ordering options | Pricing strategy and outdated technology for ordering |

## ↕️ Design thinking is bottom-up — a SWOT usually happens top-down as coming from what bosses think

When comparing what is being heard from customers with what the pizza shop owner, we have good news! There's a rough match with the SWOT analysis. And so let's proceed to unpack the business in light of feeling like we're on the right track.

### 🔖 Reminder: The SWOT analysis

| **Strengths**                                     | **Weaknesses**                                               |
| --- | --- |
| Unique garlic pizza recipe that wins top awards  | High staff turnover                                          |
| Owner trained in Sicily at some of the best pizzerias                          | Floods in the area damaged the seating areas that are in need of repair  |
| Strong local reputation                           | Absence of popular calzones from menu                        |
| Prime location on university campus               | Negative reviews from younger demographic for lack of hip ingredients |
| **Opportunities**                                 | **Threats**                                                  |
| Untapped catering potential                       | Rising competition from cheaper pizza businesses nearby |
| Growing local tech startup community              | There's nearby street construction that will impact foot traffic |
| Unexplored online presence and order capabilities | Rising cost of cheese                                        |
| Upcoming annual food fair                         | No immediate local regulatory changes but it's election season |

## 🤷 So what?

You may have noticed by now that we're not trying to dazzle you with LLM gymnastics and are instead staying in touch with the pizza shop owner's problems. Why does that matter? Because all the technowizardry in the world won't matter unless you can reliably solve a customer's unique set of problems. The deeper you can steep in the problem domain, the more easily all the AI solutions that are bursting from your fingertips can start to have meaning. 